In [ ]:
import datetime

import pandas as pd
from rembox_integration_tools import REMboxDataQuery
from rembox_integration_tools.rembox_analysis import StudyColumn, SeriesColumn


CLIENT_ID_ENV_VAR = "REMBOX_INT_CLIENT_ID"
CLIENT_PWD_ENV_VAR = "REMBOX_INT_CLIENT_PWD"
TOKEN_URI = "https://autoqa.vll.se/dpqaauth/connect/token"
API_URI = "https://rembox.vll.se/api"
ORIGIN_URI = "https://rembox.vll.se"
ANALYSIS_RULE_ID = "SomeAnalysisRuleId"

rembox = REMboxDataQuery(
    client_id_environment_variable=CLIENT_ID_ENV_VAR,
    client_secret_environment_variable=CLIENT_PWD_ENV_VAR,
    token_uri=TOKEN_URI,
    api_uri=API_URI,
    origin_uri=ORIGIN_URI,
    verify_ssl_cert=False,
)

valid_study_columns = StudyColumn()
valid_series_columns = SeriesColumn()

In [ ]:
from datetime import datetime, UTC, timedelta
from typing import Literal
from typing_extensions import Self
from uuid import UUID

from pydantic import BaseModel, model_validator


class PlotTolerance(BaseModel):
    min: float
    max: float


class AnalysisPlotTrace(BaseModel):
    traceName: str
    x: list[str | float | datetime]
    y: list[str | float | bool ]
    tolerance: PlotTolerance | None
    plotMarker: None | Literal[
        "circle", "circle-open",
        "square", "square-open",
        "diamond", "diamond-open",
        "x", "x-open",
        "triangle-up", "triangle-up-open",
        "triangle-left", "triangle-left-open",
        "triangle-right", "triangle-right-open",
        "triangle-down", "triangle-down-open",
        "star", "star-open",
        "hexagram", "hexagram-open",
    ]
    plotType: Literal["line", "scatter", "bar"]


class AnalysisResultJson(BaseModel):
    analysisResultType: Literal["TimeSeries", "Categorized", "Float", "Bool"]
    accessionNumber: str
    studyInstanceUid: str
    analysisDateTime: datetime
    analysisResultValue: float | str | bool
    withinTolerance: bool
    analysisPlotTraces: list[AnalysisPlotTrace]


class AnalysisResult(BaseModel):
    analysisRuleId: UUID
    resultWithinTolerance: bool
    analysisResultJson: AnalysisResultJson


class RemBoxAnalysisResult(BaseModel):
    analysisResult: AnalysisResult
    sendNotification: bool
    notificationMessage: str | None

    @model_validator(mode="after")
    def check_notification_message_content(self) -> Self:
        if self.sendNotification and not self.notificationMessage.strip():
            raise ValueError("Notification must be specified message when SendNotification is True")

        return self

class TimeTrigger(BaseModel):
    startDate: datetime | None
    endDate: datetime | None
    minDate: datetime | None
    selectedQuickOption: str | None


class AnalysisFilter(BaseModel):
    studyTimeInterval: TimeTrigger | None
    studyId: int | None
    studyType: str | list[str]
    machineType: str | list[str]
    machines: list[int] | None
    studyDescriptions: list[str] | None
    acquisitionProtocol: list[str] | None



class AnalysisSettings(BaseModel):
    jobMessage: str
    analysisFilter: AnalysisFilter


class RemBoxAnalysisRule(BaseModel):
    id: UUID
    displayName: str
    description: str
    analysisSettingsJson: AnalysisSettings
    created: datetime
    modified: datetime



In [ ]:
analysis_rule = rembox.get_analysis_rule_by_id(ANALYSIS_RULE_ID)

In [ ]:
analysis_rule

In [ ]:
def get_data_from_REMbox(rembox: REMboxDataQuery) -> tuple[pd.DataFrame, pd.DataFrame]:
    valid_study_columns = StudyColumn()
    valid_series_columns = SeriesColumn()
    
    # Rax at NUS
    rembox.filter_options.set_inclusive_tags(
        machine_types=["DX"],
    )
    
    current_datetime = datetime.datetime.now(datetime.UTC)
    start_time = current_datetime - datetime.timedelta(days=7)
    
    # about four month data
    rembox.filter_options.study_time_interval_start_date = f"{start_time.strftime('%Y-%m-%d')}T00:00:00Z"
    rembox.filter_options.study_time_interval_end_date = f"{current_datetime.strftime('%Y-%m-%d')}T23:59:59Z"

    rembox.add_columns(
        columns=[
            valid_study_columns.StudyDateTime,
            valid_study_columns.StudyInstanceUID,
            valid_study_columns.AccessionNumber,
            valid_study_columns.StudyId,
            valid_study_columns.Machine,
            valid_study_columns.DoseAreaProductTotal,
            valid_study_columns.TotalNumberOfIrradiationEvents,
            valid_study_columns.TotalNumberOfRadiographicFrames,
        ]
    )

    return rembox.run_query()

In [ ]:
def dap_meter_check(rule: RemBoxAnalysisRule) -> list[RemBoxAnalysisResult]:
    CLIENT_ID_ENV_VAR = "REMBOX_INT_CLIENT_ID"
    CLIENT_PWD_ENV_VAR = "REMBOX_INT_CLIENT_PWD"
    TOKEN_URI = "https://autoqa.vll.se/dpqaauth/connect/token"
    API_URI = "https://rembox.vll.se/api"
    ORIGIN_URI = "https://rembox.vll.se"

    rembox = REMboxDataQuery(
        client_id_environment_variable=CLIENT_ID_ENV_VAR,
        client_secret_environment_variable=CLIENT_PWD_ENV_VAR,
        token_uri=TOKEN_URI,
        api_uri=API_URI,
        origin_uri=ORIGIN_URI,
        verify_ssl_cert=False
    )

    valid_study_columns = StudyColumn()
    valid_series_columns = SeriesColumn()

    def get_data_from_REMbox(rembox: REMboxDataQuery) -> tuple[pd.DataFrame, pd.DataFrame]:
        # Rax at NUS
        rembox.filter_options.set_inclusive_tags(
            machine_types=[mt] if isinstance((mt := rule.analysisSettingsJson.analysisFilter.machineType), str) else mt,
        )

        current_datetime = datetime.now(UTC)
        start_time = current_datetime - timedelta(days=7)

        # about four month data
        rembox.filter_options.study_time_interval_start_date = f"{start_time.strftime('%Y-%m-%d')}T00:00:00Z"
        rembox.filter_options.study_time_interval_end_date = f"{current_datetime.strftime('%Y-%m-%d')}T23:59:59Z"

        rembox.add_columns(
            columns=[
                valid_study_columns.StudyDateTime,
                valid_study_columns.StudyInstanceUID,
                valid_study_columns.AccessionNumber,
                valid_study_columns.StudyId,
                valid_study_columns.Machine,
                valid_study_columns.DoseAreaProductTotal,
                valid_study_columns.TotalNumberOfIrradiationEvents,
                valid_study_columns.TotalNumberOfRadiographicFrames,
            ]
        )

        return rembox.run_query()

    study, series_data = get_data_from_REMbox(rembox)

    result = study[valid_study_columns.DoseAreaProductTotal].min()
    triggering_studies = study[study[valid_study_columns.DoseAreaProductTotal] <= 0]

    plot_markers = ["circle", "square", "diamond", "cross", "x", "triangle", "pentagon", "hexagram", "star", "hourglass", "bowtie", "asterisk"]

    plot_traces = [{
        "traceName": machine,
        "x": study[study[valid_study_columns.Machine] == machine][valid_study_columns.StudyDateTime].tolist(),
        "y": study[study[valid_study_columns.Machine] == machine][valid_study_columns.DoseAreaProductTotal].tolist(),
        "tolerance": None,
        "plotMarker": f"{plot_markers[ind // len(plot_markers)]}-open",
        "plotType": "scatter",
    } for ind, machine in enumerate(study[valid_study_columns.Machine].unique())]

    outputs = []

    previous_machine = ""
    message_sent_for_machine = False

    for ind, machine in enumerate(triggering_studies[valid_study_columns.Machine].unique()):
        if machine != previous_machine:
            message_sent_for_machine = False
        outputs.append(RemBoxAnalysisResult(**{
            "analysisResult": {
                "analysisRuleId": "00000000-0000-0000-0000-000000000000",  # Ersätts med ID från jobbets meddelande
                "resultWithinTolerance": result <= 0,
                "analysisResultJson": {
                    "analysisResultType": "Categorized",
                    "accessionNumber": "ettLitenAccessionNumber",
                    "studyInstanceUid": study[valid_study_columns.StudyInstanceUID][(study[valid_study_columns.Machine] == machine) & (study[valid_study_columns.DoseAreaProductTotal] <= 1)].values[0],
                    "analysisDateTime": "0001-01-01T00:00:00Z",
                    "analysisResultValue": machine,
                    "withinTolerance": False,
                    "analysisPlotTraces": plot_traces,
                    "plotType": "scatter"
                }
            },
            "sendNotification": not message_sent_for_machine,
            "notificationMessage": f"{machine} har DAP-värde som indikerar trasig DAP-mätare"
        }))

        message_sent_for_machine = True

    return outputs



In [ ]:
test_regel = RemBoxAnalysisRule(**analysis_rule)

resultat = dap_meter_check(rule=test_regel)

In [ ]:
for res in resultat:
    print(f"{res.analysisResult.analysisResultJson.analysisResultValue}: {res.analysisResult.resultWithinTolerance}")

In [ ]:
study_data, _ = get_data_from_REMbox(rembox=rembox)

In [ ]:
study = study_data.copy()

In [ ]:
result = study[valid_study_columns.DoseAreaProductTotal].min()
triggering_studies = study[study[valid_study_columns.DoseAreaProductTotal] <= 1]

plot_markers = ["circle", "square", "diamond", "cross", "x", "triangle", "pentagon", "hexagram", "star", "hourglass", "bowtie", "asterisk"]

plot_traces = [{
    "traceName": machine,
    "x": study[study[valid_study_columns.Machine] == machine][valid_study_columns.StudyDateTime].tolist(),
    "y": study[study[valid_study_columns.Machine] == machine][valid_study_columns.DoseAreaProductTotal].tolist(),
    "tolerance": None,
    "plotMarker": f"{plot_markers[ind // len(plot_markers)]}-open"
} for ind, machine in enumerate(study[valid_study_columns.Machine].unique())]

outputs = []

for ind, machine in enumerate(triggering_studies[valid_study_columns.Machine].unique()):
    outputs.append({
        "analysisResult": {
            "analysisRuleId": "00000000-0000-0000-0000-000000000000",  # Ersätts med ID från jobbets meddelande
            "resultWithinTolerance": result <= 0,
            "analysisResultJson": {
                "analysisResultType": "string",
                "accessionNumber": study[valid_study_columns.AccessionNumber][(study[valid_study_columns.Machine] == machine) & (study[valid_study_columns.DoseAreaProductTotal] <= 1)].values[0],
                "studyInstanceUid": study[valid_study_columns.StudyInstanceUID][(study[valid_study_columns.Machine] == machine) & (study[valid_study_columns.DoseAreaProductTotal] <= 1)].values[0],
                "analysisDateTime": "0001-01-01T00:00:00Z",
                "analysisResultValue": machine,
                "withinTolerance": False,
                "analysisResultPlotTraces": plot_traces,
                "plotType": "scatter"
            }
        },
        "sendNotification": ind == 0,
        "notificationMessage": f"{machine} har DAP-värde som indikerar trasig DAP-mätare"
    })

In [ ]:
print(outputs)

In [ ]:
import json
json.dumps(outputs[0])